# CHAPTER 3. Top 10 List

- ( 문자열키,  숫자값 ) 쌍인 데이터 있을때, 상위N 개를 뽑는 문제
- 예, 문자열키가 URL이고 숫자값이 방문자수 일때, 최근 일주일간에 방문자수가 상위 10개인 URL은 어떤것인가???


![](sparkr_03_01.jpg)

### Spark Top 10 Solution

### Step 3: Connect to Spark master

In [18]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))

sc <- sparkR.init(master="local[3]" )
sqlContext <- sparkRSQL.init(sc)

Re-using existing Spark Context. Please stop SparkR with sparkR.stop() or restart R to create a new Spark Context


### Step 4: Read input file from HDFS and create RDD

In [19]:
lines <- SparkR:::textFile(sc, 'top10data.txt')

In [20]:
SparkR:::take(lines, 20)

[[1]]
[1] "cat1,12"

[[2]]
[1] "cat2,13"

[[3]]
[1] "cat3,14"

[[4]]
[1] "cat4,15"

[[5]]
[1] "cat5,10"

[[6]]
[1] "cat100,100"

[[7]]
[1] "cat200,200"

[[8]]
[1] "cat300,300"

[[9]]
[1] "cat1001,1001"

[[10]]
[1] "cat67,67"

[[11]]
[1] "cat22,22"

[[12]]
[1] "cat23,23"

[[13]]
[1] "cat1000,1000"

[[14]]
[1] "cat2000,2000"

[[15]]
[1] "cat400,400"

[[16]]
[1] "cat500,500"

### Step 5 : key-value 쌍 만들기

In [44]:
make_pairs <- function ( line ) {
    tokens <- strsplit(line, ",")[[1]]
    value <- as.numeric( tokens[2] )
    result <- list( list( tokens[1], value  ) )
    return ( result )
}

In [45]:
make_pairs( 'cat400,400' )

[[1]]
[[1]][[1]]
[1] "cat400"

[[1]][[2]]
[1] 400

In [46]:
pairs <- SparkR:::flatMap( lines, make_pairs  )

In [47]:
SparkR:::take(pairs, 10)

[[1]]
[[1]][[1]]
[1] "cat1"

[[1]][[2]]
[1] 12


[[2]]
[[2]][[1]]
[1] "cat2"

[[2]][[2]]
[1] 13


[[3]]
[[3]][[1]]
[1] "cat3"

[[3]][[2]]
[1] 14


[[4]]
[[4]][[1]]
[1] "cat4"

[[4]][[2]]
[1] 15


[[5]]
[[5]][[1]]
[1] "cat5"

[[5]][[2]]
[1] 10


[[6]]
[[6]][[1]]
[1] "cat100"

[[6]][[2]]
[1] 100


[[7]]
[[7]][[1]]
[1] "cat200"

[[7]][[2]]
[1] 200


[[8]]
[[8]][[1]]
[1] "cat300"

[[8]][[2]]
[1] 300


[[9]]
[[9]][[1]]
[1] "cat1001"

[[9]][[2]]
[1] 1001


[[10]]
[[10]][[1]]
[1] "cat67"

[[10]][[2]]
[1] 67

### Step 6: Create a local top 10 list for each input partition

In [48]:
SparkR:::getNumPartitions( pairs   )

[1] 2

In [49]:
a <- list(-5,"help")
b <- list(3, "stack")
c <- list(1, "me")
d <- list(10, "overflow")

list.of.lists <- list(d,c,b,a)
list.of.lists
list.of.lists[order(sapply(list.of.lists,'[[',1))]

[[1]]
[[1]][[1]]
[1] 10

[[1]][[2]]
[1] "overflow"


[[2]]
[[2]][[1]]
[1] 1

[[2]][[2]]
[1] "me"


[[3]]
[[3]][[1]]
[1] 3

[[3]][[2]]
[1] "stack"


[[4]]
[[4]][[1]]
[1] -5

[[4]][[2]]
[1] "help"

[[1]]
[[1]][[1]]
[1] -5

[[1]][[2]]
[1] "help"


[[2]]
[[2]][[1]]
[1] 1

[[2]][[2]]
[1] "me"


[[3]]
[[3]][[1]]
[1] 3

[[3]][[2]]
[1] "stack"


[[4]]
[[4]][[1]]
[1] 10

[[4]][[2]]
[1] "overflow"

In [66]:
local_top_10 <- function( list.of.lists ) {
    list.of.lists[order(sapply(list.of.lists,'[[',1))]
    result <- head(list.of.lists, n = 3)
    return ( result )
}

In [67]:
partitions <- SparkR:::mapPartitions( pairs, local_top_10 )

In [68]:
SparkR:::take(partitions, 30)

[[1]]
[[1]][[1]]
[1] "cat1"

[[1]][[2]]
[1] 12


[[2]]
[[2]][[1]]
[1] "cat2"

[[2]][[2]]
[1] 13


[[3]]
[[3]][[1]]
[1] "cat3"

[[3]][[2]]
[1] 14


[[4]]
[[4]][[1]]
[1] "cat67"

[[4]][[2]]
[1] 67


[[5]]
[[5]][[1]]
[1] "cat22"

[[5]][[2]]
[1] 22


[[6]]
[[6]][[1]]
[1] "cat23"

[[6]][[2]]
[1] 23

### Step 7: Create the final top 10 list using collect()

In [60]:
alltop10 <- collect( partitions )

In [61]:
alltop10

[[1]]
[[1]][[1]]
[1] "cat1"

[[1]][[2]]
[1] 12


[[2]]
[[2]][[1]]
[1] "cat2"

[[2]][[2]]
[1] 13


[[3]]
[[3]][[1]]
[1] "cat3"

[[3]][[2]]
[1] 14


[[4]]
[[4]][[1]]
[1] "cat4"

[[4]][[2]]
[1] 15


[[5]]
[[5]][[1]]
[1] "cat5"

[[5]][[2]]
[1] 10


[[6]]
[[6]][[1]]
[1] "cat67"

[[6]][[2]]
[1] 67


[[7]]
[[7]][[1]]
[1] "cat22"

[[7]][[2]]
[1] 22


[[8]]
[[8]][[1]]
[1] "cat23"

[[8]][[2]]
[1] 23


[[9]]
[[9]][[1]]
[1] "cat1000"

[[9]][[2]]
[1] 1000


[[10]]
[[10]][[1]]
[1] "cat2000"

[[10]][[2]]
[1] 2000